# Logging and Monitoring Basics with tf.contrib.learn

https://www.tensorflow.org/get_started/monitors

https://github.com/tensorflow/tensorflow/blob/v1.0.0/tensorflow/examples/tutorials/monitors/iris_monitors.py

In [1]:
import tensorflow as tf
import numpy as np

tf.logging.set_verbosity(tf.logging.INFO)

tf.VERSION

'1.5.0'

**Load the Iris CSV data to TensorFlow**

In [2]:
import os
import shutil

HOME_DIR = 'iris'
DATA_DIR = os.path.join(HOME_DIR, 'data')

IRIS_TRAINING = 'iris_training.csv'
IRIS_TEST = 'iris_test.csv'

TRAINING_FILE = os.path.join(DATA_DIR, IRIS_TRAINING)
TEST_FILE = os.path.join(DATA_DIR, IRIS_TEST)

from tensorflow.contrib.learn.python.learn.datasets.base import maybe_download
maybe_download(IRIS_TRAINING, DATA_DIR, 'http://download.tensorflow.org/data/iris_training.csv')
maybe_download(IRIS_TEST, DATA_DIR, 'http://download.tensorflow.org/data/iris_test.csv')

MODEL_DIR = os.path.join(HOME_DIR, 'model_log')

if os.path.isdir(MODEL_DIR):
    shutil.rmtree(MODEL_DIR)

In [3]:
training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=TRAINING_FILE,
    target_dtype=np.int,
    features_dtype=np.float32)

test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=TEST_FILE,
    target_dtype=np.int,
    features_dtype=np.float32)

**Construct a Deep Neural Network Classifier**

In [4]:
feature_columns = [tf.contrib.layers.real_valued_column("", dimension=4)]

classifier = tf.contrib.learn.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[10, 20, 10],
    n_classes=3,
    model_dir=MODEL_DIR,
    config=tf.contrib.learn.RunConfig(save_checkpoints_secs=1)) #to ensure a sufficient number of checkpoints

INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7f10a40b00>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 1, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'iris/model_log'}


**Fit the DNNClassifier to the Iris Training Data**

In [5]:
#Customizing the Evaluation Metrics
from tensorflow.contrib.learn.python.learn.metric_spec import MetricSpec

validation_metrics = {
    'accuracy':
        MetricSpec(
            metric_fn=tf.contrib.metrics.streaming_accuracy,
            prediction_key='classes'),
    'precision':
        MetricSpec(
            metric_fn=tf.contrib.metrics.streaming_precision,
            prediction_key='classes'),
    'recall':
        MetricSpec(
            metric_fn=tf.contrib.metrics.streaming_recall,
            prediction_key='classes')
}

validation_monitor = tf.contrib.learn.monitors.ValidationMonitor(
    test_set.data,
    test_set.target,
    every_n_steps=50,
    metrics=validation_metrics,
    early_stopping_metric="loss",
    early_stopping_metric_minimize=True,
    early_stopping_rounds=200)

classifier.fit(
    x=training_set.data,
    y=training_set.target,
    steps=2000,
    monitors=[validation_monitor])

Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into iris/model_log/model.ckpt.
INFO:tensorflow:loss = 2.57684, step = 1
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_siz

DNNClassifier(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._MultiClassHead object at 0x7f7f10a40ac8>, 'hidden_units': [10, 20, 10], 'feature_columns': (_RealValuedColumn(column_name='', dimension=4, default_value=None, dtype=tf.float32, normalizer=None),), 'optimizer': None, 'activation_fn': <function relu at 0x7f7f1f9e97b8>, 'dropout': None, 'gradient_clip_norm': None, 'embedding_lr_multipliers': None, 'input_layer_min_slice_size': None})

**Evaluate Model Accuracy**

In [6]:
accuracy_score = classifier.evaluate(x=test_set.data,y=test_set.target)['accuracy']
print('Accuracy: {0:f}'.format(accuracy_score))

INFO:tensorflow:Starting evaluation at 2018-02-13-15:14:03
INFO:tensorflow:Restoring parameters from iris/model_log/model.ckpt-1150
INFO:tensorflow:Finished evaluation at 2018-02-13-15:14:03
INFO:tensorflow:Saving dict for global step 1150: accuracy = 0.93333334, global_step = 1150, loss = 0.07633651
Accuracy: 0.933333


In [7]:
new_samples = np.array([[6.4, 3.2, 4.5, 1.5], [5.8, 3.1, 5.0, 1.7]], dtype=float)
resp = classifier.predict(new_samples)
for y in resp:
    print('Predictions: {}'.format(y))

Instructions for updating:
Please switch to predict_classes, or set `outputs` argument.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Restoring parameters from iris/model_log/model.ckpt-1150
Predictions: 1
Predictions: 1
